In [9]:


import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [10]:
table = pd.read_excel('CP012104_1_-_RAST.xls', sheet_name='Sheet1')

In [55]:
driver = webdriver.Chrome()
driver.wait = WebDriverWait(driver, 5)
driver.get(f"https://blast.ncbi.nlm.nih.gov/"
           f"Blast.cgi?PROGRAM=blastp&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome")

box = driver.wait.until(EC.presence_of_element_located(
            (By.NAME, "QUERY")))
button = driver.wait.until(EC.element_to_be_clickable(
            (By.CLASS_NAME, "blastbutton")))
organism = driver.wait.until(EC.presence_of_element_located(
            (By.NAME, "EQ_MENU")))
add_options = driver.wait.until(EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="algPar"]/span[2]')))
psi_blast = driver.wait.until(EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="psiBlast"]')))

box.send_keys(table.aa_sequence[1])
organism.send_keys(f'Viruses (taxid:10239)')
psi_blast.click()
#add_options.click()
button.click()
input() # !!!Тут надо подумать как грамотно сделать паузу!!!
table_result = driver.wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="dscTable"]')))

45


In [56]:
html_table = table_result.get_attribute('outerHTML')

In [57]:
table_results = pd.read_html(html_table)[0]

In [58]:
table_results[table_results['E value'] < 0.001]

,Select for downloading or viewing reports,Description,Max score,Total score,Query cover,E value,Per. Ident,Accession,Select for PSI blast,Used to build PSSM,Newly added


In [59]:
msg = []
if len(table_results) == 0:
    msg.append('-')
else:
    table_cut = table_results[:20][
        (~table_results.Description.str.contains('hypothetical protein', regex=False))
    ]
    
    if len(table_cut) == 0:
        msg.append('hp')
    else:
        for idx, row in table_results.iterrows():
            if (
                (('hypothetical protein' not in row['Description'])
                 and ("conserved hypothetical protein" not in row['Description'])
                 and ("unnamed protein product" not in row['Description'])
                ) 
                and len(msg) < 3):
                
                msg.append(f"{row['Description']};{row['E value']}")

In [60]:
msg

['-']